In [3]:
'ㅉ' in '짱 구'

False

In [4]:
# https://hello-yeonji-stock.streamlit.app/

# 표준 라이브러리
import datetime
from io import BytesIO

# 서드파티 라이브러리
import streamlit as st
import pandas as pd
import FinanceDataReader as fdr
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib 
import koreanize_matplotlib

# 캐싱: 인자가 바뀌지 않는 함수 실행 결과를 저장 후 재사용
@st.cache_data # streamlit에서 사용할 것임
def get_krx_company_list() -> pd.DataFrame:
    """
    KRX(한국거래소) 상장 기업의 회사명과 종목코드 정보를 DataFrame으로 반환합니다.

    Returns:
        pd.DataFrame: '회사명', '종목코드' 컬럼을 가진 DataFrame
    """
    krx_df = fdr.StockListing('KRX')
    company_df = krx_df[['Name', 'Code']].rename(columns={'Name': '회사명', 'Code': '종목코드'})
    return company_df


def get_stock_code_by_company(company_name: str) -> str:
    """
    회사명을 입력받아 해당 회사의 종목코드를 반환합니다.
    Args:
        company_name (str): 조회할 회사명
    Returns:
        str: 종목코드. 입력된 회사명이 없으면 ValueError 발생
    """
    company_df = get_krx_company_list()
    codes = company_df[company_df['회사명'] == company_name]['종목코드'].values
    if len(codes) > 0:
        return codes[0]
    else:
        raise ValueError(f"'{company_name}'에 해당하는 종목코드가 없습니다.")

'''
streamlit에서 사용될 코드

def sidebar_inputs() -> tuple[str, tuple[datetime.date, datetime.date], bool]:
    """
    Streamlit 사이드바에 회사명 입력창, 날짜 선택 위젯, 확인 버튼을 생성하고 입력값을 반환합니다.

    Returns:
        tuple: (회사명(str), (시작일, 종료일)(tuple of date), 확인버튼 클릭여부(bool))
    """
    company_name = st.sidebar.text_input('회사 이름을 입력하세요: ')
    today = datetime.datetime.now()
    this_year = today.year
    jan_1 = datetime.date(this_year, 1, 1)
    selected_dates = st.sidebar.date_input(
        "시작일과 종료일을 입력하세요",
        (jan_1, today),
        None,
        today,
        format="MM.DD.YYYY",
    )
    st.sidebar.write(selected_dates)
    confirm_btn = st.sidebar.button('확인')
    return company_name, selected_dates, confirm_btn

# 우리가 필요로하는 코드조각들
stock_code = get_stock_code_by_company(company_name)
start_date = selected_dates[0].strftime(r"%Y-%m-%d")
end_date = (selected_dates[1] + datetime.timedelta(days=1)).strftime(r"%Y-%m-%d")
price_df = fdr.DataReader(f'KRX:{stock_code}', start_date, end_date)

excel_data = BytesIO()
price_df.to_excel(excel_data)
st.download_button("엑셀 파일 다운로드", excel_data, file_name='stock_data.xlsx')

'''


ModuleNotFoundError: No module named 'distutils'

In [13]:
import requests
import json
import xml.etree.ElementTree as ET # XML 응답을 처리할 경우 필요

# 1. API 기본 정보 설정
#BASE_URL = "https://www.work24.go.kr/cm/openApi/call/hr/callOpenApiSvcInfo310L01.do"
# 2. 필수 요청 Parameters 설정 [2]
# !!! [인증키] 부분은 반드시 본인이 발급받은 실제 인증키로 대체해야 합니다.
# 대괄호([])는 입력하지 않습니다.
params = {
    "authKey": "4a606afd-193d-4881-a83d-510c5d586276",
    "returnType": "JSON",  # XML 또는 JSON 선택. 여기서는 JSON으로 예시 [2]
    "outType": "1",        # '1': 리스트, '2': 상세 [2]
    "pageNum": "1",        # 시작 페이지 번호 (기본값 1, 최대 1000) [2]
    "pageSize": "100",      # 페이지당 출력 건수 (기본값 10, 최대 100) [2]
    "srchTraStDt": "20230101", # 훈련 시작일의 시작 범위 (YYYYMMDD 형식) [2]
    "srchTraEndDt": "20231231", # 훈련 시작일의 종료 범위 (YYYYMMDD 형식) [2]
    "sort": "ASC",         # 정렬 방법: "ASC" (오름차순), "DESC" (내림차순) [2]
    "sortCol": "2"         # 정렬 기준 컬럼: 1(훈련기관명), 2(훈련시작일), 3(취업률), 4(만족도점수) [2]
}
BASE_URL = f"https://www.work24.go.kr/cm/openApi/call/hr/callOpenApiSvcInfo310L01.do?authKey={params['authKey']}&returnType=XML"


print(BASE_URL)
# 3. 선택 요청 Parameters 추가 [2]
# 예시: 훈련지역 대분류 '서울' (코드 '11') 및 훈련유형 'K-디지털 트레이닝' (코드 'C0104') 추가
# 여러 선택조건을 조합하여 사용할 수 있습니다.
optional_params = {
    "srchTraArea1": "11",   # 훈련지역 대분류: '11' 서울 [2]
    "crseTracseSe": "C0104" # 훈련유형: 'C0104' K-디지털 트레이닝 [2]
    # "srchNcs1": "20",       # NCS 직종 1차분류 코드: '20' 정보통신 [2]
    # "srchTraProcessNm": "파이썬", # 훈련과정명 검색 [2]
}

# 필수 및 선택 Parameters를 병합
params.update(optional_params)

# 4. API 호출
print("API 호출 중...")
try:
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status() # HTTP 에러가 발생하면 예외를 발생시킵니다.

    # 5. 응답 데이터 처리
    if params["returnType"] == "JSON":
        data = response.json()
        print("\nAPI 호출 성공 (JSON):")
        print(data)
        # JSON 데이터 구조에 따라 접근
        if data and 'HRDNet' in data and 'srchList' in data['HRDNet']:
            # 검색된 총 건수, 현재 페이지, 페이지당 출력 개수 정보 [3]
            total_count = data['HRDNet'].get('scn_cnt', 'N/A')
            current_page = data['HRDNet'].get('pageNum', 'N/A')
            page_size = data['HRDNet'].get('pageSize', 'N/A')

            print(f"총 검색 건수: {total_count}개 [3]")
            print(f"현재 페이지: {current_page}, 페이지당 출력 개수: {page_size}개 [3]")

            # 훈련과정 목록 출력 [3]
            training_list = data['HRDNet']['srchList'].get('scn_list', [])
            if not isinstance(training_list, list): # 단일 항목일 경우 리스트가 아닌 딕셔너리로 올 수 있음
                training_list = [training_list]

            if training_list:
                print("\n검색된 훈련과정 목록:")
                for i, course in enumerate(training_list):
                    print(f"--- 훈련과정 {i+1} ---")
                    print(f"  제목: {course.get('TITLE', 'N/A')} [3]")
                    print(f"  훈련기관: {course.get('SUB_TITLE', 'N/A')} [3]")
                    print(f"  훈련 시작일: {course.get('TRA_START_DATE', 'N/A')} [3]")
                    print(f"  훈련 종료일: {course.get('TRA_END_DATE', 'N/A')} [3]")
                    print(f"  주소: {course.get('ADDRESS', 'N/A')} [3]")
                    print(f"  수강비: {course.get('COURSE_MAN', 'N/A')} [3]")
                    print(f"  실제 훈련비: {course.get('REAL_MAN', 'N/A')} [3]")
                    print(f"  취업률 (고용보험 3개월): {course.get('EI_EMPL_RATE3', 'N/A')}% [3]")
                    print(f"  만족도 점수: {course.get('STDG_SCOR', 'N/A')} [3]")
                    print("-" * 20)
            else:
                print("검색된 훈련과정이 없습니다.")
        else:
            print("API 응답에서 'HRDNet' 또는 'srchList'를 찾을 수 없습니다. 응답 구조를 확인하세요.")
            print(data) # 전체 응답 출력하여 구조 확인
    elif params["returnType"] == "XML":
        # XML 응답을 처리할 경우
        root = ET.fromstring(response.text)
        print("\nAPI 호출 성공 (XML):")
        # XML 응답 구조에 따라 접근 [3]
        # 예시: 총 검색 건수
        scn_cnt = root.find('scn_cnt')
        if scn_cnt is not None:
            print(f"총 검색 건수: {scn_cnt.text}개 [3]")
        
        # 훈련과정 목록 (srchList/scn_list) 순회 [3]
        for course in root.findall('.//scn_list'):
            title = course.find('TITLE')
            if title is not None:
                print(f"  제목: {title.text} [3]")
            # 필요한 다른 항목들도 위와 같은 방식으로 접근하여 출력
            
    else:
        print(f"지원하지 않는 returnType: {params['returnType']}")

except requests.exceptions.HTTPError as http_err:
    print(f"HTTP 오류 발생: {http_err}")
except requests.exceptions.ConnectionError as conn_err:
    print(f"연결 오류 발생: {conn_err}")
except requests.exceptions.Timeout as timeout_err:
    print(f"타임아웃 오류 발생: {timeout_err}")
except requests.exceptions.RequestException as req_err:
    print(f"요청 오류 발생: {req_err}")
except json.JSONDecodeError as json_err:
    print(f"JSON 디코딩 오류: {json_err}. API 응답이 유효한 JSON 형식이 아닙니다.")
    print("응답 내용:", response.text[:500]) # 응답의 일부를 출력하여 문제 확인
except Exception as e:
    print(f"예상치 못한 오류 발생: {e}")

https://www.work24.go.kr/cm/openApi/call/hr/callOpenApiSvcInfo310L01.do?authKey=4a606afd-193d-4881-a83d-510c5d586276&returnType=XML
API 호출 중...
요청 오류 발생: Expecting value: line 1 column 1 (char 0)


In [ ]:
import requests
import json
import xml.etree.ElementTree as ET # XML 응답을 처리할 경우 필요

# 1. API 기본 정보 설정
BASE_URL = "https://www.work24.go.kr/cm/openApi/call/hr/callOpenApiSvcInfo310L01.do"
# 2. 필수 요청 Parameters 설정 [2]
# !!! [인증키] 부분은 반드시 본인이 발급받은 실제 인증키로 대체해야 합니다.
# 대괄호([])는 입력하지 않습니다.
params = {
    "authKey": "4a606afd-193d-4881-a83d-510c5d586276",
    "returnType": "JSON",  # XML 또는 JSON 선택. 여기서는 JSON으로 예시 [2]
    "outType": "1",        # '1': 리스트, '2': 상세 [2]
    "pageNum": "1",        # 시작 페이지 번호 (기본값 1, 최대 1000) [2]
    "pageSize": "100",      # 페이지당 출력 건수 (기본값 10, 최대 100) [2]
    "srchTraStDt": "20230101", # 훈련 시작일의 시작 범위 (YYYYMMDD 형식) [2]
    "srchTraEndDt": "20231231", # 훈련 시작일의 종료 범위 (YYYYMMDD 형식) [2]
    "sort": "ASC",         # 정렬 방법: "ASC" (오름차순), "DESC" (내림차순) [2]
    "sortCol": "2"         # 정렬 기준 컬럼: 1(훈련기관명), 2(훈련시작일), 3(취업률), 4(만족도점수) [2]
}

#BASE_URL = f"https://www.work24.go.kr/cm/openApi/call/hr/callOpenApiSvcInfo310L01.do?authKey={params['authKey']}&returnType=XML&outType=1&pageNum=1&pageSize=20&srchTraStDt=20141001&srchTraEndDt=20141231&sort=ASC&sortCol=2"
print(BASE_URL)
# 3. 선택 요청 Parameters 추가 [2]
# 예시: 훈련지역 대분류 '서울' (코드 '11') 및 훈련유형 'K-디지털 트레이닝' (코드 'C0104') 추가
# 여러 선택조건을 조합하여 사용할 수 있습니다.

# optional_params = {
#     "srchTraArea1": "11",   # 훈련지역 대분류: '11' 서울 [2]
#     "crseTracseSe": "C0104" # 훈련유형: 'C0104' K-디지털 트레이닝 [2]
#     # "srchNcs1": "20",       # NCS 직종 1차분류 코드: '20' 정보통신 [2]
#     # "srchTraProcessNm": "파이썬", # 훈련과정명 검색 [2]
# }

# 필수 및 선택 Parameters를 병합
# params.update(optional_params)

# 4. API 호출
print("API 호출 중...")
try:
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status() # HTTP 에러가 발생하면 예외를 발생시킵니다.

    # 5. 응답 데이터 처리
    if params["returnType"] == "JSON":
        data = response.json()
        print("\nAPI 호출 성공 (JSON):")
        print(data)
        # JSON 데이터 구조에 따라 접근
        if data and 'HRDNet' in data and 'srchList' in data['HRDNet']:
            # 검색된 총 건수, 현재 페이지, 페이지당 출력 개수 정보 [3]
            total_count = data['HRDNet'].get('scn_cnt', 'N/A')
            current_page = data['HRDNet'].get('pageNum', 'N/A')
            page_size = data['HRDNet'].get('pageSize', 'N/A')

            print(f"총 검색 건수: {total_count}개 [3]")
            print(f"현재 페이지: {current_page}, 페이지당 출력 개수: {page_size}개 [3]")

            # 훈련과정 목록 출력 [3]
            training_list = data['HRDNet']['srchList'].get('scn_list', [])
            if not isinstance(training_list, list): # 단일 항목일 경우 리스트가 아닌 딕셔너리로 올 수 있음
                training_list = [training_list]

            if training_list:
                print("\n검색된 훈련과정 목록:")
                for i, course in enumerate(training_list):
                    print(f"--- 훈련과정 {i+1} ---")
                    print(f"  제목: {course.get('TITLE', 'N/A')} [3]")
                    print(f"  훈련기관: {course.get('SUB_TITLE', 'N/A')} [3]")
                    print(f"  훈련 시작일: {course.get('TRA_START_DATE', 'N/A')} [3]")
                    print(f"  훈련 종료일: {course.get('TRA_END_DATE', 'N/A')} [3]")
                    print(f"  주소: {course.get('ADDRESS', 'N/A')} [3]")
                    print(f"  수강비: {course.get('COURSE_MAN', 'N/A')} [3]")
                    print(f"  실제 훈련비: {course.get('REAL_MAN', 'N/A')} [3]")
                    print(f"  취업률 (고용보험 3개월): {course.get('EI_EMPL_RATE3', 'N/A')}% [3]")
                    print(f"  만족도 점수: {course.get('STDG_SCOR', 'N/A')} [3]")
                    print("-" * 20)
            else:
                print("검색된 훈련과정이 없습니다.")
        else:
            print("API 응답에서 'HRDNet' 또는 'srchList'를 찾을 수 없습니다. 응답 구조를 확인하세요.")
            print(data) # 전체 응답 출력하여 구조 확인
    elif params["returnType"] == "XML":
        # XML 응답을 처리할 경우
        root = ET.fromstring(response.text)
        print("\nAPI 호출 성공 (XML):")
        # XML 응답 구조에 따라 접근 [3]
        # 예시: 총 검색 건수
        scn_cnt = root.find('scn_cnt')
        if scn_cnt is not None:
            print(f"총 검색 건수: {scn_cnt.text}개 [3]")
        
        # 훈련과정 목록 (srchList/scn_list) 순회 [3]
        for course in root.findall('.//scn_list'):
            title = course.find('TITLE')
            if title is not None:
                print(f"  제목: {title.text} [3]")
            # 필요한 다른 항목들도 위와 같은 방식으로 접근하여 출력
            
    else:
        print(f"지원하지 않는 returnType: {params['returnType']}")

except requests.exceptions.HTTPError as http_err:
    print(f"HTTP 오류 발생: {http_err}")
except requests.exceptions.ConnectionError as conn_err:
    print(f"연결 오류 발생: {conn_err}")
except requests.exceptions.Timeout as timeout_err:
    print(f"타임아웃 오류 발생: {timeout_err}")
except requests.exceptions.RequestException as req_err:
    print(f"요청 오류 발생: {req_err}")
except json.JSONDecodeError as json_err:
    print(f"JSON 디코딩 오류: {json_err}. API 응답이 유효한 JSON 형식이 아닙니다.")
    print("응답 내용:", response.text[:500]) # 응답의 일부를 출력하여 문제 확인
except Exception as e:
    print(f"예상치 못한 오류 발생: {e}")

https://www.work24.go.kr/cm/openApi/call/hr/callOpenApiSvcInfo310L01.do
API 호출 중...

API 호출 성공 (JSON):
{'srchList': [{'eiEmplCnt3Gt10': '', 'eiEmplRate6': '', 'eiEmplCnt3': '', 'eiEmplRate3': '', 'certificate': '', 'title': '모든것을뛰어넘는이기는비즈니스전략', 'realMan': '73780', 'telNo': '02-2163-5724', 'stdgScor': '0', 'traStartDate': '2023-01-01', 'grade': '', 'ncsCd': '02010101', 'regCourseMan': '0', 'trprDegr': '22', 'address': '서울 성동구', 'traEndDate': '2023-02-28', 'subTitle': '(주)알파코', 'instCd': '200400456', 'trngAreaCd': '11200', 'trprId': 'ACG20213000806323', 'yardMan': '2500', 'courseMan': '73780', 'trainTarget': '국민내일배움카드(일반)', 'trainTargetCd': 'C0061', 'trainstCstId': '500020014456', 'contents': '', 'subTitleLink': 'https://hrd.work24.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=ACG20213000806323&tracseTme=22&trainstCstmrId=500020014456&crseTracseSe=C0031', 'titleLink': 'https://hrd.work24.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=ACG20213000806323&tracseTme=22&trainstCstmrId=500020014456&cr

In [16]:
import requests
import json
import xml.etree.ElementTree as ET # XML 응답을 처리할 경우 필요

# 1. API 기본 정보 설정
#BASE_URL = "https://www.work24.go.kr/cm/openApi/call/hr/callOpenApiSvcInfo310L01.do"
# 2. 필수 요청 Parameters 설정 [2]
# !!! [인증키] 부분은 반드시 본인이 발급받은 실제 인증키로 대체해야 합니다.
# 대괄호([])는 입력하지 않습니다.
params = {
    "authKey": "4a606afd-193d-4881-a83d-510c5d586276", # 4a606afd-193d-4881-a83d-510c5d586276
    "returnType": "JSON",  # XML 또는 JSON 선택. 여기서는 JSON으로 예시 [2]
    "outType": "1",        # '1': 리스트, '2': 상세 [2]
    "pageNum": "1",        # 시작 페이지 번호 (기본값 1, 최대 1000) [2]
    "pageSize": "100",      # 페이지당 출력 건수 (기본값 10, 최대 100) [2]
    "srchTraStDt": "20250101", # 훈련 시작일의 시작 범위 (YYYYMMDD 형식) [2]
    "srchTraEndDt": "20250716", # 훈련 시작일의 종료 범위 (YYYYMMDD 형식) [2]
    "sort": "ASC",         # 정렬 방법: "ASC" (오름차순), "DESC" (내림차순) [2]
    "sortCol": "2"         # 정렬 기준 컬럼: 1(훈련기관명), 2(훈련시작일), 3(취업률), 4(만족도점수) [2]
}

BASE_URL = f"https://www.work24.go.kr/cm/openApi/call/hr/callOpenApiSvcInfo310L01.do"
# 3. 선택 요청 Parameters 추가 [2]
# 예시: 훈련지역 대분류 '서울' (코드 '11') 및 훈련유형 'K-디지털 트레이닝' (코드 'C0104') 추가
# 여러 선택조건을 조합하여 사용할 수 있습니다.
# optional_params = {
#     "srchTraArea1": "11",   # 훈련지역 대분류: '11' 서울 [2]
#     "crseTracseSe": "C0104" # 훈련유형: 'C0104' K-디지털 트레이닝 [2]
#     # "srchNcs1": "20",       # NCS 직종 1차분류 코드: '20' 정보통신 [2]
#     # "srchTraProcessNm": "파이썬", # 훈련과정명 검색 [2]
# }

# # 필수 및 선택 Parameters를 병합
# params.update(optional_params)

# 4. API 호출
print("API 호출 중...")
try:
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status() # HTTP 에러가 발생하면 예외를 발생시킵니다.
    print(response.text)

    # 5. 응답 데이터 처리
    if params["returnType"] == "JSON":
        data = response.json()
        print("\nAPI 호출 성공 (JSON):")
        print(data)
        # JSON 데이터 구조에 따라 접근
        if data and 'HRDNet' in data and 'srchList' in data['HRDNet']:
            # 검색된 총 건수, 현재 페이지, 페이지당 출력 개수 정보 [3]
            total_count = data['HRDNet'].get('scn_cnt', 'N/A')
            current_page = data['HRDNet'].get('pageNum', 'N/A')
            page_size = data['HRDNet'].get('pageSize', 'N/A')

            print(f"총 검색 건수: {total_count}개 [3]")
            print(f"현재 페이지: {current_page}, 페이지당 출력 개수: {page_size}개 [3]")

            # 훈련과정 목록 출력 [3]
            training_list = data['HRDNet']['srchList'].get('scn_list', [])
            if not isinstance(training_list, list): # 단일 항목일 경우 리스트가 아닌 딕셔너리로 올 수 있음
                training_list = [training_list]

            if training_list:
                print("\n검색된 훈련과정 목록:")
                for i, course in enumerate(training_list):
                    print(f"--- 훈련과정 {i+1} ---")
                    print(f"  제목: {course.get('TITLE', 'N/A')} [3]")
                    print(f"  훈련기관: {course.get('SUB_TITLE', 'N/A')} [3]")
                    print(f"  훈련 시작일: {course.get('TRA_START_DATE', 'N/A')} [3]")
                    print(f"  훈련 종료일: {course.get('TRA_END_DATE', 'N/A')} [3]")
                    print(f"  주소: {course.get('ADDRESS', 'N/A')} [3]")
                    print(f"  수강비: {course.get('COURSE_MAN', 'N/A')} [3]")
                    print(f"  실제 훈련비: {course.get('REAL_MAN', 'N/A')} [3]")
                    print(f"  취업률 (고용보험 3개월): {course.get('EI_EMPL_RATE3', 'N/A')}% [3]")
                    print(f"  만족도 점수: {course.get('STDG_SCOR', 'N/A')} [3]")
                    print("-" * 20)
            else:
                print("검색된 훈련과정이 없습니다.")
        else:
            print("API 응답에서 'HRDNet' 또는 'srchList'를 찾을 수 없습니다. 응답 구조를 확인하세요.")
            print(data) # 전체 응답 출력하여 구조 확인
    elif params["returnType"] == "XML":
        # XML 응답을 처리할 경우
        root = ET.fromstring(response.text)
        print("\nAPI 호출 성공 (XML):")
        # XML 응답 구조에 따라 접근 [3]
        # 예시: 총 검색 건수
        scn_cnt = root.find('scn_cnt')
        if scn_cnt is not None:
            print(f"총 검색 건수: {scn_cnt.text}개 [3]")
        
        # 훈련과정 목록 (srchList/scn_list) 순회 [3]
        for course in root.findall('.//scn_list'):
            title = course.find('TITLE')
            if title is not None:
                print(f"  제목: {title.text} [3]")
            # 필요한 다른 항목들도 위와 같은 방식으로 접근하여 출력
            
    else:
        print(f"지원하지 않는 returnType: {params['returnType']}")

except requests.exceptions.HTTPError as http_err:
    print(f"HTTP 오류 발생: {http_err}")
except requests.exceptions.ConnectionError as conn_err:
    print(f"연결 오류 발생: {conn_err}")
except requests.exceptions.Timeout as timeout_err:
    print(f"타임아웃 오류 발생: {timeout_err}")
except requests.exceptions.RequestException as req_err:
    print(f"요청 오류 발생: {req_err}")
except json.JSONDecodeError as json_err:
    print(f"JSON 디코딩 오류: {json_err}. API 응답이 유효한 JSON 형식이 아닙니다.")
    print("응답 내용:", response.text[:500]) # 응답의 일부를 출력하여 문제 확인
except Exception as e:
    print(f"예상치 못한 오류 발생: {e}")

API 호출 중...
{"srchList":[{"eiEmplCnt3Gt10":"","eiEmplRate6":"","eiEmplCnt3":"","eiEmplRate3":"","certificate":"","title":"당장 시작해야 하는 영업비밀과 지식재산권","realMan":"87780","telNo":"031-1644-9874","stdgScor":"0","traStartDate":"2025-01-01","grade":"","ncsCd":"05010201","regCourseMan":"0","trprDegr":"24","address":"경기 수원시 영통구","traEndDate":"2025-01-31","subTitle":"이룸캠퍼스","instCd":"201700192","trngAreaCd":"41117","trprId":"ACG20243000972854","yardMan":"1000","courseMan":"87780","trainTarget":"국민내일배움카드(일반)","trainTargetCd":"C0061","trainstCstId":"500020055230","contents":"","subTitleLink":"https://hrd.work24.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=ACG20243000972854&tracseTme=24&trainstCstmrId=500020055230&crseTracseSe=C0031","titleLink":"https://hrd.work24.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=ACG20243000972854&tracseTme=24&trainstCstmrId=500020055230&crseTracseSe=C0031","titleIcon":"<img src=\"hrd.work24.go.kr/images/ico_system13.gif\" alt=\"근로자직업능력개발훈련\" title=\"근로자직업능력개발훈련\" />"},{"eiEm

In [18]:
import requests
import json
import xml.etree.ElementTree as ET # XML 응답을 처리할 경우 필요

# 1. API 기본 정보 설정
#BASE_URL = "https://www.work24.go.kr/cm/openApi/call/hr/callOpenApiSvcInfo310L01.do"
# 2. 필수 요청 Parameters 설정 [2]
# !!! [인증키] 부분은 반드시 본인이 발급받은 실제 인증키로 대체해야 합니다.
# 대괄호([])는 입력하지 않습니다.
params = {
    "authKey": "4a606afd-193d-4881-a83d-510c5d586276", # 4a606afd-193d-4881-a83d-510c5d586276
    "returnType": "JSON",  # XML 또는 JSON 선택. 여기서는 JSON으로 예시 [2]
    "outType": "1",        # '1': 리스트, '2': 상세 [2]
    "pageNum": "1",        # 시작 페이지 번호 (기본값 1, 최대 1000) [2]
    "pageSize": "100",      # 페이지당 출력 건수 (기본값 10, 최대 100) [2]
    "srchTraStDt": "20250101", # 훈련 시작일의 시작 범위 (YYYYMMDD 형식) [2]
    "srchTraEndDt": "20250716", # 훈련 시작일의 종료 범위 (YYYYMMDD 형식) [2]
    "sort": "ASC",         # 정렬 방법: "ASC" (오름차순), "DESC" (내림차순) [2]
    "sortCol": "2"         # 정렬 기준 컬럼: 1(훈련기관명), 2(훈련시작일), 3(취업률), 4(만족도점수) [2]
}

BASE_URL = f"https://www.work24.go.kr/cm/openApi/call/hr/callOpenApiSvcInfo310L01.do"
# 3. 선택 요청 Parameters 추가 [2]
# 예시: 훈련지역 대분류 '서울' (코드 '11') 및 훈련유형 'K-디지털 트레이닝' (코드 'C0104') 추가
# 여러 선택조건을 조합하여 사용할 수 있습니다.
optional_params = {
    "srchTraArea1": "11",   # 훈련지역 대분류: '11' 서울 [2]
    "crseTracseSe": "C0104" # 훈련유형: 'C0104' K-디지털 트레이닝 [2]
    # "srchNcs1": "20",       # NCS 직종 1차분류 코드: '20' 정보통신 [2]
    # "srchTraProcessNm": "파이썬", # 훈련과정명 검색 [2]
}

# 필수 및 선택 Parameters를 병합
params.update(optional_params)

# 4. API 호출
print("API 호출 중...")
try:
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status() # HTTP 에러가 발생하면 예외를 발생시킵니다.

    # 5. 응답 데이터 처리
    if params["returnType"] == "JSON":
        data = response.json()
        print("\nAPI 호출 성공 (JSON):")
        print(type(data), data.keys())
        # JSON 데이터 구조에 따라 접근
        if data and 'srchList' in data:
            # 검색된 총 건수, 현재 페이지, 페이지당 출력 개수 정보 [3]
            total_count = data.get('scn_cnt', 'N/A')
            current_page = data.get('pageNum', 'N/A')
            page_size = data.get('pageSize', 'N/A')

            print(f"총 검색 건수: {total_count}개 [3]")
            print(f"현재 페이지: {current_page}, 페이지당 출력 개수: {page_size}개 [3]")
            print('---------------------------------------------------------')
            print(type(data['srchList']), data['srchList'][0], len( data['srchList']))

            # 훈련과정 목록 출력 [3]
            training_list = data['srchList']
            if not isinstance(training_list, list): # 단일 항목일 경우 리스트가 아닌 딕셔너리로 올 수 있음
                training_list = [training_list]

            if training_list:
                print("\n검색된 훈련과정 목록:")
                for i, course in enumerate(training_list):
                    print(f"--- 훈련과정 {i+1} ---")
                    print(f"  제목: {course.get('title', 'N/A')} [3]")
                    print(f"  훈련기관: {course.get('subTitle', 'N/A')} [3]")
                    print(f"  훈련 시작일: {course.get('traStartDate', 'N/A')} [3]")
                    print(f"  훈련 종료일: {course.get('traEndDate', 'N/A')} [3]")
                    print(f"  주소: {course.get('address', 'N/A')} [3]")
                    print(f"  수강비: {course.get('courseMan', 'N/A')} [3]")
                    print(f"  실제 훈련비: {course.get('realMan', 'N/A')} [3]")
                    print(f"  취업률 (고용보험 3개월): {course.get('eIEmplRate3', 'N/A')}% [3]")
                    print(f"  만족도 점수: {course.get('stdgScor', 'N/A')} [3]")
                    print("-" * 20)
            else:
                print("검색된 훈련과정이 없습니다.")
        else:
            print("API 응답에서 'HRDNet' 또는 'srchList'를 찾을 수 없습니다. 응답 구조를 확인하세요.")
            print(data) # 전체 응답 출력하여 구조 확인
    elif params["returnType"] == "XML":
        # XML 응답을 처리할 경우
        root = ET.fromstring(response.text)
        print("\nAPI 호출 성공 (XML):")
        # XML 응답 구조에 따라 접근 [3]
        # 예시: 총 검색 건수
        scn_cnt = root.find('scn_cnt')
        if scn_cnt is not None:
            print(f"총 검색 건수: {scn_cnt.text}개 [3]")
        
        # 훈련과정 목록 (srchList/scn_list) 순회 [3]
        for course in root.findall('.//scn_list'):
            title = course.find('TITLE')
            if title is not None:
                print(f"  제목: {title.text} [3]")
            # 필요한 다른 항목들도 위와 같은 방식으로 접근하여 출력
            
    else:
        print(f"지원하지 않는 returnType: {params['returnType']}")

except requests.exceptions.HTTPError as http_err:
    print(f"HTTP 오류 발생: {http_err}")
except requests.exceptions.ConnectionError as conn_err:
    print(f"연결 오류 발생: {conn_err}")
except requests.exceptions.Timeout as timeout_err:
    print(f"타임아웃 오류 발생: {timeout_err}")
except requests.exceptions.RequestException as req_err:
    print(f"요청 오류 발생: {req_err}")
except json.JSONDecodeError as json_err:
    print(f"JSON 디코딩 오류: {json_err}. API 응답이 유효한 JSON 형식이 아닙니다.")
    print("응답 내용:", response.text[:500]) # 응답의 일부를 출력하여 문제 확인
except Exception as e:
    print(f"예상치 못한 오류 발생: {e}")

API 호출 중...

API 호출 성공 (JSON):
<class 'dict'> dict_keys(['srchList', 'scn_cnt', 'pageSize', 'pageNum'])
총 검색 건수: 658개 [3]
현재 페이지: 1, 페이지당 출력 개수: 100개 [3]
---------------------------------------------------------
<class 'list'> {'eiEmplCnt3Gt10': '', 'eiEmplRate6': '', 'eiEmplCnt3': '', 'eiEmplRate3': '', 'certificate': '', 'title': '자바 스프링 리액트로 완성하는 클라우드 활용 풀스택 개발 ', 'realMan': '9477000', 'telNo': '02-855-9611', 'stdgScor': '92.2', 'traStartDate': '2025-01-02', 'grade': '', 'ncsCd': '20010202', 'regCourseMan': '19', 'trprDegr': '16', 'address': '서울 관악구', 'traEndDate': '2025-09-25', 'subTitle': '한국정보교육원(구.경원직업전문학교)', 'instCd': '200400659', 'trngAreaCd': '11350', 'trprId': 'AIG20230000412579', 'yardMan': '25', 'courseMan': '9477000', 'trainTarget': 'K-디지털 트레이닝', 'trainTargetCd': 'C0104', 'trainstCstId': '500020031803', 'contents': '', 'subTitleLink': 'https://hrd.work24.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20230000412579&tracseTme=16&trainstCstmrId=500020031803&crseTracseSe=C006

In [19]:
training_list

[{'eiEmplCnt3Gt10': '',
  'eiEmplRate6': '',
  'eiEmplCnt3': '',
  'eiEmplRate3': '',
  'certificate': '',
  'title': '자바 스프링 리액트로 완성하는 클라우드 활용 풀스택 개발 ',
  'realMan': '9477000',
  'telNo': '02-855-9611',
  'stdgScor': '92.2',
  'traStartDate': '2025-01-02',
  'grade': '',
  'ncsCd': '20010202',
  'regCourseMan': '19',
  'trprDegr': '16',
  'address': '서울 관악구',
  'traEndDate': '2025-09-25',
  'subTitle': '한국정보교육원(구.경원직업전문학교)',
  'instCd': '200400659',
  'trngAreaCd': '11350',
  'trprId': 'AIG20230000412579',
  'yardMan': '25',
  'courseMan': '9477000',
  'trainTarget': 'K-디지털 트레이닝',
  'trainTargetCd': 'C0104',
  'trainstCstId': '500020031803',
  'contents': '',
  'subTitleLink': 'https://hrd.work24.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20230000412579&tracseTme=16&trainstCstmrId=500020031803&crseTracseSe=C0061',
  'titleLink': 'https://hrd.work24.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20230000412579&tracseTme=16&trainstCstmrId=500020031803&crseTracseSe=C0061',
  'titleIcon

In [2]:
import pandas as pd

df=pd.read_csv('./훈련과정_전체데이터.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: './훈련과정_전체데이터.csv'